In [ ]:
import pandas as pd
import sys
import numpy as np
from itertools import combinations
pd.set_option('display.max_rows', 100)

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# CSV file with PreRedacted Data

In [ ]:
file_path = r'..\data\UserData2.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)

# Instantiate the anonymizer

In [ ]:
sensitive_list = ['Subgroup1', 'Subgroup2']

In [ ]:
frequency_value = 'Counts'

In [ ]:
anonymizer = DataAnonymizer(df_data, sensitive_columns=sensitive_list, frequency=frequency_value)

In [ ]:
df_log = anonymizer.create_log()

In [ ]:
df_log.to_csv('Log.csv')

In [ ]:
df_less_than_threshold = anonymizer.less_than_threshold_not_zero()

In [ ]:
df_less_than_threshold.to_csv('LessThanThreshold.csv')

In [ ]:
df_one_count = anonymizer.one_count_redacted()

In [ ]:
df_one_count.to_csv('OneCountRedact.csv')

In [ ]:
sensitive_combinations = sorted([combo for i in range(1, len(sensitive_list) + 1) for combo in combinations(sensitive_list, i)], key=len, reverse=True)

In [ ]:
df_one_count_na = df_one_count.copy()

In [ ]:
temp_value = 'NaFill'

In [ ]:
for sensitive_column in sensitive_list:
    df_one_count_na[sensitive_column].fillna(temp_value, inplace=True)

In [ ]:
for sensitive_combination in sensitive_combinations:
    list_combination = list(sensitive_combination)
    if list_combination != sensitive_list:
        string_combination = ''.join(list_combination)
        df_redact_less = df_one_count_na[df_one_count_na['RedactBinary'] == 1]
        df_redact_less['Redacted'] = 1
        df_count = df_redact_less.groupby(['Grouping'] + list_combination)['Redacted'].count().reset_index()
        df_one_redacted = df_count[df_count['Redacted'] == 1]
        if not df_one_redacted.empty:
            df_not_redacted = df_one_count_na[df_one_count_na['RedactBinary'] != 1]
            df_minimum = df_not_redacted.groupby(['Grouping'] + list_combination, dropna=False)['Counts'].min().reset_index()
            df_minimum.rename(columns={'Counts':'LastMiniumValue'}, inplace=True)
            df_minimum_redacted = df_one_redacted.merge(df_minimum, on = ['Grouping'] + list_combination)
            df_minimum_one = df_one_count_na.merge(df_minimum_redacted, on = ['Grouping'] + list_combination, how='left')
            mask = (df_minimum_one['Counts'] == df_minimum_one['LastMiniumValue'])
            df_one_count_na.loc[mask, 'RedactBinary'] = 1
            df_one_count_na.loc[mask, 'Redact'] = 'Secondary Suppression'
        # df_primary = df_one_count.merge(df_redact_merge, on=['Grouping'] + list_combination, how='left')
        # mask = (df_primary['Redacted'] > 1) & (df_primary["MinimumValue" + string_combination] == df_primary[self.frequency])
        # self.df_log.loc[mask, 'RedactBinary'] = 1
        # self.df_log.loc[mask, 'Redact'] = 'Secondary Suppression'

In [ ]:
df_not_redacted.loc[df_one_count_na['RedactBinary'] == 1, 'RedactBinary'] = 1
df_not_redacted.loc[df_one_count_na['Redact'] == 'Secondary Suppression', 'Redact'] = 'Secondary Suppression'

In [ ]:
df_not_redacted.to_csv('RemainingRedaction.csv')

In [ ]:
df_sum_redact = anonymizer.sum_redact()

In [ ]:
display(df_sum_redact)

In [ ]:
df_zero = anonymizer.one_redact_zero()

In [ ]:
df_one_count.to_csv('OneRedactZero.csv')

In [ ]:
df_cross_suppression = anonymizer.cross_suppression()

In [ ]:
display(df_cross_suppression)

In [ ]:
df_apply_log = anonymizer.apply_log()

In [ ]:
display(df_apply_log)

In [ ]:
df_apply_log.to_csv('MergedTotals.csv')